#**Scenario: Employee Work Data for a Tech Company**

**Step 1: Prepare Data in PySpark**

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("EmployeeWorkData").getOrCreate()
spark

In [22]:
from pyspark.sql import Row
data = [
Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine", Salary=95000, HoursPerWeek=42),
Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform", Salary=87000, HoursPerWeek=45),
Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch", Salary=65000, HoursPerWeek=40),
Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach", Salary=70000, HoursPerWeek=38),
Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine", Salary=99000, HoursPerWeek=48),
Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media", Salary=62000, HoursPerWeek=35),
Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp", Salary=58000, HoursPerWeek=37),
Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000, HoursPerWeek=41),
Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite", Salary=91000, HoursPerWeek=46),
Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000, HoursPerWeek=36)
]

df = spark.createDataFrame(data)
df.show()

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



**Step 2: Create Views**

Create a Local Temp View

In [5]:
df.createOrReplaceTempView("employees_local")

Create a Global Temp View

In [6]:
df.createOrReplaceGlobalTempView("employees_global")

**Part A: Exercises on Local View ( employees_local )**

1. List all employees working on the "AI Engine" project.

In [8]:
spark.sql("SELECT * FROM employees_local WHERE Project = 'AI Engine'").show()

+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+



2. Show all employees from the "Marketing" department with salaries greater than 60,000.

In [9]:
spark.sql("SELECT * FROM employees_local WHERE Department = 'Marketing' AND Salary >60000").show()

+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+



3. Calculate the average salary for each department.

In [12]:
spark.sql("SELECT Department, AVG(Salary) AS avg_sal FROM employees_local GROUP BY Department").show()

+-----------+-------+
| Department|avg_sal|
+-----------+-------+
|      Sales|71500.0|
|Engineering|93000.0|
|  Marketing|63500.0|
|         HR|59000.0|
+-----------+-------+



4. List the top 3 highest paid employees overall.

In [14]:
spark.sql("SELECT * FROM employees_local ORDER BY Salary DESC LIMIT 3").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



5. Find employees who work more than 40 hours per week.

In [15]:
spark.sql("SELECT * FROM employees_local WHERE HoursPerWeek > 40").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



6. Group by project and display the number of employees per project.

In [19]:
spark.sql("SELECT Project, COUNT(EmpID) AS No_of_emp FROM employees_local GROUP BY Project").show()

+---------------+---------+
|        Project|No_of_emp|
+---------------+---------+
|  Data Platform|        1|
|      AI Engine|        2|
| Product Launch|        1|
|Client Outreach|        1|
| Security Suite|        1|
|  Policy Revamp|        1|
|       Lead Gen|        1|
|   Social Media|        1|
|     Onboarding|        1|
+---------------+---------+



7. Drop the local view. Try querying again — what happens?

In [21]:
spark.sql("DROP VIEW employees_local")

spark.sql("SELECT * FROM employees_local").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`employees_local` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

**Part B: Exercises on Global View ( employees_global )**

1. Retrieve all "HR" employees working fewer than 38 hours/week.

In [26]:
new_spark = SparkSession.builder.appName("EmployeeWorkData1").getOrCreate()

df.createOrReplaceGlobalTempView("employees_global")

new_spark.sql("SELECT * FROM global_temp.employees_global WHERE Department = 'HR' AND HoursPerWeek < 38").show()

+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+



2. Calculate the total salary payout for each department.

In [28]:
new_spark.sql("SELECT Department, SUM(Salary) AS Tot_sal FROM global_temp.employees_global GROUP BY Department").show()

+-----------+-------+
| Department|Tot_sal|
+-----------+-------+
|      Sales| 143000|
|Engineering| 372000|
|  Marketing| 127000|
|         HR| 118000|
+-----------+-------+



3. For each employee, add a derived column Status :

If HoursPerWeek > 45 → 'Overworked'
Otherwise → 'Normal'

In [30]:
new_spark.sql("SELECT *, CASE WHEN HoursPerWeek > 45 THEN 'Overworked' ELSE 'Normal' END AS Status FROM global_temp.employees_global").show()

+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|Overworked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|Overworked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

4. Count the total number of employees working on each "Project" .

In [31]:
new_spark.sql("SELECT COUNT(EmpID) AS no_of_emp, Project FROM global_temp.employees_global GROUP BY Project").show()

+---------+---------------+
|no_of_emp|        Project|
+---------+---------------+
|        1|  Data Platform|
|        2|      AI Engine|
|        1| Product Launch|
|        1|Client Outreach|
|        1| Security Suite|
|        1|  Policy Revamp|
|        1|       Lead Gen|
|        1|   Social Media|
|        1|     Onboarding|
+---------+---------------+



5. List employees whose salary is above the average salary in their department.

In [32]:
new_spark.sql("SELECT * FROM global_temp.employees_global WHERE Salary > (SELECT AVG(Salary) FROM global_temp.employees_global)").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



6. Open a new Spark session and query "global_temp.employees_global" from there.

In [33]:
new_spark1 = SparkSession.builder.appName("EmployeeWorkData2").getOrCreate()

new_spark1.sql("SELECT * FROM global_temp.employees_global").show()

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



**Bonus Challenges**

1. Use a window function to assign rank to employees within each department based on salary.

In [34]:
new_spark.sql("SELECT *, RANK() OVER (PARTITION BY Department ORDER BY Salary DESC) AS rank FROM global_temp.employees_global").show()

+-----+-----+-----------+---------------+------+------------+----+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|rank|
+-----+-----+-----------+---------------+------+------------+----+
|  105|Divya|Engineering|      AI Engine| 99000|          48|   1|
|  101| Ravi|Engineering|      AI Engine| 95000|          42|   2|
|  109| Neha|Engineering| Security Suite| 91000|          46|   3|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|   4|
|  110|Farah|         HR|     Onboarding| 60000|          36|   1|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|   2|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|   1|
|  106| Amit|  Marketing|   Social Media| 62000|          35|   2|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|   1|
|  104|Anita|      Sales|Client Outreach| 70000|          38|   2|
+-----+-----+-----------+---------------+------+------------+----+



2. Create another view (local or global) that only contains "Engineering" employees.

In [36]:
df.createOrReplaceTempView("employees_local1")

spark.sql("SELECT * FROM employees_local1 WHERE Department = 'Engineering'").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



3. Create a SQL view that filters out all employees working < 38 hours and saves it as "active_employees" .

In [39]:
new_spark.sql("CREATE OR REPLACE TEMP VIEW active_employees AS SELECT * FROM global_temp.employees_global WHERE HoursPerWeek >= 38;").show()
new_spark.sql("SELECT * FROM active_employees").show()

++
||
++
++

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
+-----+-----+-----------+---------------+------+------------+

